In [1]:
# !pip install --quiet --pre --upgrade dgl-cu101
# !pip install --quiet torch==1.6.0

In [3]:
import os
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GATConv
# from bipartite_gatconv import BipartiteGATConv
from collections import defaultdict
from tqdm import tqdm
tqdm().pandas()
import pickle
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
import warnings
warnings.filterwarnings('ignore')

data_dir="/workspace/cjiang/eagle_project/CAP_graph/dataset/"
root_dir="/workspace/cjiang/eagle_project/CAP_graph/CAP_without_zipcode/"

os.chdir(root_dir)

print("{:<20}{:<20}".format("torch version",torch.__version__))
print("{:<20}{:<20}".format("DGL version",dgl.__version__))


0it [00:00, ?it/s]

torch version       1.6.0               
DGL version         0.6a210131          


#### Load Dataset

In [4]:
def read_csv(data_dir, file):
    start=time.time()
    df=pd.read_csv(os.path.join(data_dir,file))
    df.drop_duplicates(inplace=True)
    end=time.time()
    print("Dataloading running time is {:0.4f}".format(end-start))
    print("The Shape of Dataset is {}".format(df.shape))
    return df

def to_pickle(data_dir,file_in,file_out):
    start=time.time()
    file_in.to_pickle(os.path.join(data_dir,file_out))
    end=time.time()
    print("pickle time is {:0.4f}".format(end-start))
    
def read_pickle(data_dir,file):
    start=time.time()
    df=pd.read_pickle(os.path.join(data_dir,file))
    end=time.time()
    print("loading time is {:0.4f}".format(end-start))
    print("The Shape of Dataset is {}".format(df.shape))
    return df

In [5]:
# Pers_Edges=read_csv(data_dir, 'Pers_Edges.csv')
# to_pickle(data_dir,Pers_Edges,'Pers_Edges_pickle')
Pers_Edges=read_pickle(data_dir,'Pers_Edges_pickle')

loading time is 5.2505
The Shape of Dataset is (45782901, 3)


In [6]:
# Busi_Edges=read_csv(data_dir, 'Busi_Edges.csv')
# to_pickle(data_dir,Busi_Edges,'Busi_Edges_pickle')
Busi_Edges=read_pickle(data_dir,'Busi_Edges_pickle')

loading time is 1.8708
The Shape of Dataset is (20819095, 3)


In [5]:
# zipcode_Edges=read_csv(data_dir, 'zipcode_Edges.csv')
# to_pickle(data_dir,zipcode_Edges,'zipcode_Edges_pickle')
# zipcode_Edges=read_pickle(data_dir,'zipcode_Edges_pickle')

In [7]:
# Product_Edges=read_csv(data_dir, 'Product_Edges.csv')
# to_pickle(data_dir,Product_Edges,'Product_Edges_pickle')
Product_Edges=read_pickle(data_dir,'Product_Edges_pickle')

loading time is 4.4591
The Shape of Dataset is (35635860, 3)


In [8]:
# vertices=read_csv(data_dir, 'vertices_clean.csv')
# to_pickle(data_dir,vertices,'vertices_pickle')
vertices=read_pickle(data_dir,'vertices_pickle')

loading time is 35.1528
The Shape of Dataset is (27884332, 29)


#### Transform product edge data and create product bit vector

In [9]:
Product_Edges=Product_Edges[Product_Edges["dst"].isin(["P_AUTO","P_HOME","P_RENT"])]

In [10]:
def map_prod(LIST):
    if (LIST.count("P_AUTO")==1) and (LIST.count("P_HOME")==1) and (LIST.count("P_RENT")==1):
        return [1,1,1,"111"]
    elif (LIST.count("P_AUTO")==1) and (LIST.count("P_HOME")==1) and (LIST.count("P_RENT")==0):
        return [1,1,0,"110"]
    elif (LIST.count("P_AUTO")==1) and (LIST.count("P_HOME")==0) and (LIST.count("P_RENT")==1):
        return [1,0,1,"101"]
    elif (LIST.count("P_AUTO")==0) and (LIST.count("P_HOME")==1) and (LIST.count("P_RENT")==1):
        return [0,1,1,"011"]
    elif (LIST.count("P_AUTO")==1) and (LIST.count("P_HOME")==0) and (LIST.count("P_RENT")==0):
        return [1,0,0,"100"]
    elif (LIST.count("P_AUTO")==0) and (LIST.count("P_HOME")==1) and (LIST.count("P_RENT")==0):
        return [0,1,0,"010"]
    elif (LIST.count("P_AUTO")==0) and (LIST.count("P_HOME")==0) and (LIST.count("P_RENT")==1):
        return [0,0,1,"001"]
    elif (LIST.count("P_AUTO")==0) and (LIST.count("P_HOME")==0) and (LIST.count("P_RENT")==0):
        return [0,0,0,"000"]

In [11]:
# import numba as nb
# @nb.jit(nopython=True)
def Flatten_Product(Product_Edges):
    Product_Edges.sort_values(by="src",inplace=True)
    src, _, dst=np.array(Product_Edges).transpose()
    row=np.unique(src).shape[0]
    col=np.unique(dst).shape[0]
    Flatten_df=np.zeros([row,col+2],dtype=int)   ### usaa_number,  P_AUTO,  P_HOME,  P_RENT,  Product_bit_Vector
    
    uniq_v, uniq_e=np.unique(Product_Edges["src"],return_inverse=True)
    _, idx=np.unique(uniq_e,return_index=True)

    for i in tqdm(np.arange(len(uniq_v))):
        
        if i!=len(uniq_v)-1:
            Flatten_df[i][0]=list(set(Product_Edges[idx[i]:idx[i+1]].src.astype(int)))[0]
            Flatten_df[i][1:5]=map_prod(list(set(Product_Edges[idx[i]:idx[i+1]].dst)))
        else:
            Flatten_df[i][0]=list(set(Product_Edges[idx[i]:].src.astype(int)))[0]
            Flatten_df[i][1:5]=map_prod(list(set(Product_Edges[idx[i]:].dst)))
    
    Flatten_df=pd.DataFrame(Flatten_df,columns=["USAA_Number","P_AUTO","P_HOME","P_RENT","Prod_Trace"])
    Flatten_df["Prod_Trace"]=Flatten_df["Prod_Trace"].apply(lambda x : '{0:0>3}'.format(x)) ## add the leading zero in some prod_trace
    
    return Flatten_df

In [12]:
Flatten_df=Flatten_Product(Product_Edges)

100%|██████████| 7383666/7383666 [1:22:07<00:00, 1498.40it/s]


In [9]:
# to_pickle(data_dir,Flatten_df,'Flatten_Product_pickle')
Flatten_Product_pickle=read_pickle(data_dir,'Flatten_Product_pickle')

loading time is 1.4688
The Shape of Dataset is (7383666, 5)


In [9]:
tempt1=pd.DataFrame(Flatten_Product_pickle["Prod_Trace"].value_counts()).reset_index().rename(columns={'index':'product','Prod_Trace':'count'})
tempt2=pd.DataFrame(Flatten_Product_pickle["Prod_Trace"].value_counts(normalize=True)).reset_index().rename(columns={'index':'product','Prod_Trace':'percentage'})
tempt1.merge(tempt2, on="product", how="inner").style.format({'count':'{:,}','percentage':'{:.2%}'})

,product,count,percentage
0,110,"2,938,245",39.79%
1,100,"2,435,787",32.99%
2,101,"1,461,264",19.79%
3,001,"262,009",3.55%
4,010,"226,935",3.07%
5,111,"56,659",0.77%
6,011,"2,767",0.04%


In [10]:
print("Pers_Edges Dataframe:")
print(Pers_Edges.head(2))
print()
print("Busi_Edges Dataframe:")
print(Busi_Edges.head(2))
# print()
# print("zipcode_Edges Dataframe:")
# print(zipcode_Edges.head(2))
print()
print("Product_Edges Dataframe:")
print(Flatten_Product_pickle.head(2))

Pers_Edges Dataframe:
        src     rel       dst
0  50777622  Parent  53997959
1  50777622  Parent  57122393

Busi_Edges Dataframe:
        src             rel       dst
0  50777622  Busi_rel_Other  57122393
1  50777622         SPONSOR  53995390

Product_Edges Dataframe:
   USAA_Number  P_AUTO  P_HOME  P_RENT Prod_Trace
0     50777622       1       1       0        110
1     50777628       1       1       0        110


#### Create bi-directional relationship

In [10]:
def datashow(dataframe):
    tempt1=pd.DataFrame(dataframe.rel.value_counts()).reset_index().rename(columns={'index':'rels','rel':'count'})
    tempt2=pd.DataFrame(dataframe.rel.value_counts(normalize=True)).reset_index().rename(columns={'index':'rels','rel':'percentage'})
    return tempt1.merge(tempt2, on="rels", how="inner").style.format({'count':'{:,}','percentage':'{:.2%}'})

def make_bidirectional(dataframe, rel_types, rev_rel_types):
    """
    dataframe :    triplet(src, rel, dst)
    rel_types:     relationship to be inversed bidirectiohally
    rev_rel_types: the inversed relationship name
    """
    np_df=np.array(dataframe)
    src,rel,dst=np_df.transpose()
    rel_v2=rel.copy()
    for idx, val in enumerate(rel_types):
        rel_v2[rel==val]=rev_rel_types[idx]
        
    src,dst=np.concatenate((src,dst)), np.concatenate((dst, src))
    rel=np.concatenate((rel, rel_v2))
    
    DF=pd.DataFrame(sorted(zip(src, rel, dst)),columns=['src','rel','dst'])
    DF.drop_duplicates(inplace=True)
    
    return DF

In [12]:
datashow(Busi_Edges)

,rels,count,percentage
0,SPONSOR,"15,958,685",76.65%
1,AUTO_RELATED,"4,121,995",19.80%
2,Busi_rel_Other,"738,415",3.55%


In [13]:
rel_types=Busi_Edges.rel.unique().tolist()  ## ['Busi_rel_Other', 'SPONSOR', 'AUTO_RELATED']
rev_rel_types=['Busi_rel_Other','SPONSEE','AUTO_RELATED']
for idx, val in enumerate(rel_types):
    print("{:<30}{:<30}".format(val, rev_rel_types[idx]))

Busi_rel_Other                Busi_rel_Other                
SPONSOR                       SPONSEE                       
AUTO_RELATED                  AUTO_RELATED                  


In [11]:
%%time
rel_types=Busi_Edges.rel.unique().tolist()  ## ['Busi_rel_Other', 'SPONSOR', 'AUTO_RELATED']
rev_rel_types=['Busi_rel_Other','SPONSEE','AUTO_RELATED']
Busi_Edges_bi=make_bidirectional(Busi_Edges, rel_types, rev_rel_types)

CPU times: user 2min 28s, sys: 2min 19s, total: 4min 47s
Wall time: 4min 33s


In [15]:
datashow(Busi_Edges_bi)

,rels,count,percentage
0,SPONSEE,"15,958,685",38.93%
1,SPONSOR,"15,958,685",38.93%
2,AUTO_RELATED,"8,056,456",19.65%
3,Busi_rel_Other,"1,022,064",2.49%


In [15]:
# datashow(zipcode_Edges)
# zipcode_Edges['dst'].unique().shape[0]

# rel_types=zipcode_Edges.rel.unique().tolist()  ## ['Located_In']
# rev_rel_types=["Location_of"]
# zipcode_Edges_bi=make_bidirectional(zipcode_Edges, rel_types, rev_rel_types)

# datashow(zipcode_Edges_bi)

In [16]:
datashow(Pers_Edges)

,rels,count,percentage
0,Spouse,"15,424,898",33.69%
1,Child,"11,018,879",24.07%
2,Parent,"10,980,501",23.98%
3,Ex-Spouse,"3,622,554",7.91%
4,Pers_rel_Other,"1,829,526",4.00%
5,Step-Child,"1,148,701",2.51%
6,Step-Parent,"1,148,623",2.51%
7,Brother_Sister,"609,219",1.33%


In [12]:
rel_types=Pers_Edges.rel.unique().tolist()  
rev_rel_types=['Child','Spouse','Parent','Ex-Spouse','Pers_rel_Other','Brother_Sister','Step-Child','Step-Parent']
for idx, val in enumerate(rel_types):
    print("{:<30}{:<30}".format(val, rev_rel_types[idx]))

Parent                        Child                         
Spouse                        Spouse                        
Child                         Parent                        
Ex-Spouse                     Ex-Spouse                     
Pers_rel_Other                Pers_rel_Other                
Brother_Sister                Brother_Sister                
Step-Parent                   Step-Child                    
Step-Child                    Step-Parent                   


In [13]:
%%time
Pers_Edges_bi=make_bidirectional(Pers_Edges, rel_types, rev_rel_types)

CPU times: user 5min 56s, sys: 6min 50s, total: 12min 47s
Wall time: 12min 26s


In [19]:
datashow(Pers_Edges_bi)

,rels,count,percentage
0,Spouse,"15,485,971",33.56%
1,Child,"11,067,385",23.98%
2,Parent,"11,067,385",23.98%
3,Ex-Spouse,"3,682,614",7.98%
4,Pers_rel_Other,"1,892,856",4.10%
5,Step-Child,"1,165,991",2.53%
6,Step-Parent,"1,165,991",2.53%
7,Brother_Sister,"618,666",1.34%


#### Construct a Heterogeneous Graph

#### re-label source nodes and destination nodes for each node type

In [14]:
%%time
#### relabel the nodes of USAA Member such that they are continous integers from 0 to max
src_pers, _ ,dst_pers=np.array(Pers_Edges_bi).transpose()
src_busi, _ ,dst_busi=np.array(Busi_Edges_bi).transpose()
# src_zipcode, _ ,dst_zipcode=np.array(zipcode_Edges_bi[zipcode_Edges_bi['rel']=="Located_In"]).transpose()

all_usaanr=np.concatenate((src_pers,dst_pers,src_busi,dst_busi))
uniq_usaanr = np.unique(all_usaanr)
# uniq_zipcode = np.unique(dst_zipcode)

uniq_usaanr.sort()
# uniq_zipcode.sort()
usaanr_map = {id:idx for idx, id in enumerate(uniq_usaanr)}
# zipcode_map = {id:idx for idx, id in enumerate(uniq_zipcode)}

CPU times: user 6min 18s, sys: 35.6 s, total: 6min 54s
Wall time: 6min 52s


In [21]:
Pers_Edges_bi['src'] = list(map(usaanr_map.get, Pers_Edges_bi['src']))
Pers_Edges_bi['dst'] = list(map(usaanr_map.get, Pers_Edges_bi['dst']))
print(len(usaanr_map))
print(Pers_Edges_bi['src'].unique().max())
print(Pers_Edges_bi['dst'].unique().max())

24561910
24561909
24561909


In [22]:
Busi_Edges_bi['src'] = list(map(usaanr_map.get, Busi_Edges_bi['src']))
Busi_Edges_bi['dst'] = list(map(usaanr_map.get, Busi_Edges_bi['dst']))
print(len(usaanr_map))
print(Busi_Edges_bi['src'].unique().max())
print(Busi_Edges_bi['dst'].unique().max())

24561910
24561909
24561909


In [23]:
# zipcode_Edges_v1=zipcode_Edges_bi[zipcode_Edges_bi['rel']=="Located_In"]
# zipcode_Edges_v1['src'] = list(map(usaanr_map.get, zipcode_Edges_v1['src']))
# zipcode_Edges_v1['dst'] = list(map(zipcode_map.get, zipcode_Edges_v1['dst']))

# zipcode_Edges_v2=zipcode_Edges_bi[zipcode_Edges_bi['rel']=="Location_of"]
# zipcode_Edges_v2['src'] = list(map(zipcode_map.get, zipcode_Edges_v2['src']))
# zipcode_Edges_v2['dst'] = list(map(usaanr_map.get, zipcode_Edges_v2['dst']))

# zipcode_Edges_bi=zipcode_Edges_v1.append(zipcode_Edges_v2,ignore_index=True)
# print(len(usaanr_map))
# print(zipcode_Edges_bi['src'].unique().max())
# print(len(zipcode_map))
# print(zipcode_Edges_bi['dst'].unique().max())

#### Re-label Edges types

In [27]:
# relation_encoder=LabelEncoder()
# relation_encoder.fit(pd.concat([Pers_Edges['rel'],Busi_Edges['rel'],zipcode_Edges['rel']]))

In [24]:
%%time
pers_rel_nums = {"rel":     {"Parent": 0, "Child": 1, "Spouse": 2,"Ex-Spouse": 3,"Brother_Sister": 4, "Step-Parent": 5, "Step-Child": 6, "Pers_rel_Other": 7}}
busi_rel_nums = {"rel":     {"SPONSOR": 8, "SPONSEE": 9, "AUTO_RELATED": 10,"Busi_rel_Other":11}}
# zipcode_rel_nums = {"rel":     {"Located_In": 12,"Location_of": 13}}

# start=time.time()
Pers_Edges_bi.replace(pers_rel_nums, inplace=True)
Busi_Edges_bi.replace(busi_rel_nums, inplace=True)
# zipcode_Edges_bi.replace(zipcode_rel_nums, inplace=True)

# end=time.time()
# print("running time is {:0.4f}".format(end-start))

CPU times: user 43.7 s, sys: 5.86 s, total: 49.6 s
Wall time: 49.3 s


In [25]:
print(Pers_Edges_bi['rel'].unique())
print(Busi_Edges_bi['rel'].unique())
# print(zipcode_Edges_bi['rel'].unique())

[0 2 1 3 7 4 5 6]
[10 11  8  9]


In [26]:
data_dict=dict()

### Personal Relationship ###
rel=np.array(Pers_Edges_bi['rel'])
src_pers=np.array(Pers_Edges_bi['src'])
dst_pers=np.array(Pers_Edges_bi['dst'])

data_dict.update({('usaanr', 'Parent', 'usaanr')  :         (src_pers[rel==0], dst_pers[rel==0])})
data_dict.update({('usaanr', 'Child', 'usaanr')   :         (src_pers[rel==1], dst_pers[rel==1])})
data_dict.update({('usaanr', 'Spouse', 'usaanr')  :         (src_pers[rel==2], dst_pers[rel==2])})
data_dict.update({('usaanr', 'Ex-Spouse', 'usaanr')  :      (src_pers[rel==3], dst_pers[rel==3])})
data_dict.update({('usaanr', 'Brother_Sister', 'usaanr')  : (src_pers[rel==4], dst_pers[rel==4])})
data_dict.update({('usaanr', 'Step-Parent', 'usaanr')     : (src_pers[rel==5], dst_pers[rel==5])})
data_dict.update({('usaanr', 'Step-Child', 'usaanr')    :   (src_pers[rel==6], dst_pers[rel==6])})
data_dict.update({('usaanr', 'Pers_rel_Other', 'usaanr')  : (src_pers[rel==7], dst_pers[rel==7])})

### Business Relationship ###
rel=np.array(Busi_Edges_bi['rel'])
src_busi=np.array(Busi_Edges_bi['src'])
dst_busi=np.array(Busi_Edges_bi['dst'])

data_dict.update({('usaanr', 'SPONSOR', 'usaanr')    :      (src_busi[rel==8], dst_busi[rel==8])})
data_dict.update({('usaanr', 'SPONSEE', 'usaanr')    :      (src_busi[rel==9], dst_busi[rel==9])})
data_dict.update({('usaanr', 'AUTO_RELATED', 'usaanr')   :  (src_busi[rel==10], dst_busi[rel==10])})
data_dict.update({('usaanr', 'Busi_rel_Other', 'usaanr') :  (src_busi[rel==11], dst_busi[rel==11])})

### zipcode relationship ###
# zipcode_Edges_v1=zipcode_Edges_bi[zipcode_Edges_bi['rel']==12]
# src_v1=np.array(zipcode_Edges_v1['src'])
# dst_v1=np.array(zipcode_Edges_v1['dst'])

# zipcode_Edges_v2=zipcode_Edges_bi[zipcode_Edges_bi['rel']==13]
# src_v2=np.array(zipcode_Edges_v2['src'])
# dst_v2=np.array(zipcode_Edges_v2['dst'])

# data_dict.update({('usaanr', 'Located_In', 'zipcode')  :   (src_v1, dst_v1)})
# data_dict.update({('zipcode', 'Location_of','usaanr')  :   (src_v2, dst_v2)})

In [27]:
start=time.time()
g = dgl.heterograph(data_dict)
end=time.time()
print("running time is {:0.4f}".format(end-start))

running time is 0.6430


In [4]:
def graph_show(G):
    print('*'*50)
    print("Node_types: " , G.ntypes)
    print("Edge_types: " , G.etypes)
    print('*'*50)
    print("Canonical Etypes of Graph is:\n")
    for srctype, etype, dsttype in G.canonical_etypes:
        print("{:<20}{:<20}{:<20}".format(srctype, etype, dsttype))
    print('*'*50)
    Total_ntype_num=0
    for i in G.ntypes:
        print(f"number of ntype={i:<20}  {G.number_of_nodes(i):<15,}")
        Total_ntype_num+=G.number_of_nodes(i)
    print('*'*50)
    print("Total number of nodes is {:,}".format(Total_ntype_num))
    print('*'*50)
    Total_edge_num=0
    for j in G.etypes:
        print(f"number of etype={j:<20}  {G.number_of_edges(j):<15,}")
        Total_edge_num+=G.number_of_edges(j)
    print('*'*50)
    print("Total number of edges is {:,}".format(Total_edge_num))
    print('*'*50)
    for nty in G.ntypes:
        if G.nodes[nty].data!={}:
            print('*'*50)
            print(f"The attributes for the node type={nty}")
            print('*'*50)
            for key, scheme in G.node_attr_schemes(ntype=nty).items():
                print("{:<40}{}".format(key,G.nodes[nty].data[key].shape))

In [29]:
graph_show(g)

**************************************************
Node_types:  ['usaanr']
Edge_types:  ['AUTO_RELATED', 'Brother_Sister', 'Busi_rel_Other', 'Child', 'Ex-Spouse', 'Parent', 'Pers_rel_Other', 'SPONSEE', 'SPONSOR', 'Spouse', 'Step-Child', 'Step-Parent']
**************************************************
Canonical Etypes of Graph is:

usaanr              AUTO_RELATED        usaanr              
usaanr              Brother_Sister      usaanr              
usaanr              Busi_rel_Other      usaanr              
usaanr              Child               usaanr              
usaanr              Ex-Spouse           usaanr              
usaanr              Parent              usaanr              
usaanr              Pers_rel_Other      usaanr              
usaanr              SPONSEE             usaanr              
usaanr              SPONSOR             usaanr              
usaanr              Spouse              usaanr              
usaanr              Step-Child          usaanr          

#### Adding features/attributes to the nodes of USAA Members
The CAP dataset has some features for USAA Member.

* usaayr:  USAA Number Issue Year
* AGE / AGE BUCKET
* ORIGEL :  Original Eligibility
* ELIG2 : Current Eligibility
* cmpyelig: Company Eligibility
* Segment: Alpha ~ Juliet
* SEX
* MARST : Marriage Status
* MILST : Military Status
* MLIST_OrigStat: Original Military Status
* ENLPAYGD: Military Pay Grade
* BRANCH: Military BRANCH of Service
* ACTCORP : Corporate Active Status
* STATE
* Segment: Alpha to Juliet
* ZIPCD: zipcode


We use label encoding for all categorial variables. <br>
In addition, there is a node data "type" that indicates the node type of usaa member, zipcode in the heterogenous graph. <br>
The nodes of zipcode don't have the same features as the node of member. 

In [29]:
vertices.columns

Index(['usaanr', 'USAAISSUSDT', 'usaayr', 'AGE', 'AGE_BAND', 'ORIGEL', 'ELIG2',
       'cmpyelig', 'SEX', 'MARST', 'PERSST', 'DEATHSDT', 'BRANCH', 'MILST',
       'MLIST_OrigStat', 'enl1stsdt', 'COMMSDT', 'ENLPAYGD', 'ACTCORP',
       'ZIPCD', 'STATE', 'Segment', 'AUT_POL_ANN_PRM_AMT',
       'HO_FIRE_POL_PRM_AMT', 'REN_POL_PRM_AMT', 'UMB_POL_PRM_AMT',
       'VPP_POL_PRM_AMT', 'TOT_PRM_AMT', 'PROD_TRACE'],
      dtype='object')

In [34]:
if g.number_of_nodes(ntype="usaanr")==vertices.shape[0]:
    vertices_v2=vertices
else:
    vertices_v2=vertices[vertices['usaanr'].isin(all_usaanr)]
print("{:<20} {:<15,}".format("size of original vertices",vertices.shape[0]))
print("{:<20} {:<15,}".format("size of updated vertices",vertices_v2.shape[0]))

# assert g.number_of_nodes(ntype="usaanr")==vertices.shape[0], "the shape of feature data is not equal to the number of USAA member"

size of original vertices 27,884,332     
size of updated vertices 24,561,910     


#### sort the vertices dataframe based on the order of nodes in graph

In [36]:
vertices_v2['usaanr'] = list(map(usaanr_map.get, vertices_v2['usaanr']))
vertices_v2.sort_values(by=["usaanr"],inplace=True)

root_dir="/workspace/cjiang/eagle_project/CAP_graph/CAP_wihtout_zipcode/"
to_pickle(root_dir,vertices_v2,'vertices_reindex')
vertices=read_pickle(root_dir,'vertices_reindex')

loading time is 38.8307
The Shape of Dataset is (24561910, 29)


#### Create node type feature
The reason I create node type feature is to embedding the node USAA Member , product and zipcode separately. Unlike the node of usaanr,  the nodes of zipcode don't have other features except for node type

In [35]:
#### Nodes type feature
# g.nodes['usaanr'].data['type'] = torch.zeros(size=[g.number_of_nodes(ntype='usaanr'),1]).long()
# g.nodes['zipcode'].data['type'] = torch.ones(size=[g.number_of_nodes(ntype='zipcode'),1]).long()

In [37]:
#### Bin the numerical variable
def Bin_Numerical(args,b):
#     if args==0:
#         return str(0)
    for i in range(len(b)-1):
        if args>=b[i] and args<=b[i+1]:
#             return str((int(b[i]), int(b[i+1])))
            return int(i)

In [38]:
feat=['usaayr','AGE_BAND','ORIGEL', 'ELIG2', 'cmpyelig','SEX', 'MARST','BRANCH','ENLPAYGD','MILST',
       'MLIST_OrigStat','ACTCORP', 'STATE', 'Segment','ZIPCD']
vertices_feat=vertices.loc[:,feat]

# for col in vertices_feat:
#     if col !='usaanr':
#         vertices_feat[col]=vertices_feat[col].astype('str')
    
class_le=LabelEncoder()

for i in tqdm(range(len(vertices_feat.columns)),position=0,leave=True):
    col=vertices_feat.columns[i]
    vertices_feat[col]=vertices_feat[col].astype('str')
    vertices_feat[col]=class_le.fit_transform(vertices_feat[col])

# for col in vertices_feat.columns:
#     if vertices_feat[col].dtype=="object" and col !='usaanr':
#         vertices_feat[col]=class_le.fit_transform(vertices_feat[col])

100%|██████████| 15/15 [03:20<00:00, 13.39s/it]


In [39]:
for col in vertices_feat.columns:
    g.nodes['usaanr'].data[col]= torch.tensor( np.expand_dims(np.array(vertices_feat[col]), 1) )

#### create label

In [23]:
Flatten_Product_pickle.head(2)

,USAA_Number,P_AUTO,P_HOME,P_RENT,Prod_Trace
0,50777622,1,1,0,110
1,50777628,1,1,0,110


In [24]:
Flatten_Product_pickle.shape[0]

7383666

In [25]:
prod_df=Flatten_Product_pickle[Flatten_Product_pickle["USAA_Number"].isin(all_usaanr)]
prod_df=prod_df.loc[:,["USAA_Number","Prod_Trace"]].rename(columns={"USAA_Number":"usaanr"})
prod_df['usaanr'] = list(map(usaanr_map.get, prod_df['usaanr']))
prod_df['usaanr'].unique().max()

24561121

In [30]:
vertices=read_pickle(root_dir,'vertices_reindex')

loading time is 31.8726
The Shape of Dataset is (24561910, 29)


In [31]:
vertices_v1=vertices.merge(prod_df, on='usaanr', how="left").loc[:,['usaanr',"Prod_Trace"]]

In [32]:
vertices_v1["Prod_Trace"].value_counts(dropna=False)

NaN    17906619
110     2790185
100     2123313
101     1257366
001      222153
010      206359
111       53393
011        2522
Name: Prod_Trace, dtype: int64

In [33]:
vertices_v1.fillna(value={"Prod_Trace":"000"},inplace=True)
vertices_v1.sort_values(by=["usaanr"],inplace=True)
# vertices_v1["Prod_Trace"].value_counts(dropna=False)                          

In [34]:
tempt1=pd.DataFrame(vertices_v1["Prod_Trace"].value_counts()).reset_index().rename(columns={'index':'product','Prod_Trace':'count'})
tempt2=pd.DataFrame(vertices_v1["Prod_Trace"].value_counts(normalize=True)).reset_index().rename(columns={'index':'product','Prod_Trace':'percentage'})
tempt1.merge(tempt2, on="product", how="inner").style.format({'count':'{:,}','percentage':'{:.2%}'})

,product,count,percentage
0,000,"17,906,619",72.90%
1,110,"2,790,185",11.36%
2,100,"2,123,313",8.64%
3,101,"1,257,366",5.12%
4,001,"222,153",0.90%
5,010,"206,359",0.84%
6,111,"53,393",0.22%
7,011,"2,522",0.01%


In [47]:
Prod_Trace = vertices_v1["Prod_Trace"].unique()
Prod_Trace.sort()
product_map = {id:idx for idx, id in enumerate(Prod_Trace)}
product_map

{'000': 0,
 '001': 1,
 '010': 2,
 '011': 3,
 '100': 4,
 '101': 5,
 '110': 6,
 '111': 7}

In [48]:
vertices_v1['Prod_Trace'] = list(map(product_map.get, vertices_v1['Prod_Trace']))
node_labels=torch.tensor(np.expand_dims(np.array(vertices_v1.loc[:,"Prod_Trace"]),1))

In [28]:
prod_df.shape[0]

6655291

#### Create training(80%), validation(10%) and test(10%) mask based on each category of product

In [50]:
def mask_func(train_idx,all_idx):
    train_idx=pd.DataFrame({"idx":train_idx})
    all_idx=pd.DataFrame({"idx":all_idx})
    all_index = all_idx.set_index(['idx']).index
    train_index = train_idx.set_index(['idx']).index
    mask = all_index.isin(train_index)
    return mask

In [51]:
train_idx=[]
val_idx=[]
test_idx=[]

LABEL=node_labels.numpy().squeeze()
IDX=np.arange(LABEL.shape[0])
prod_list=np.unique(LABEL).tolist()
for i in tqdm(range(len(prod_list)),position=0,leave=True):
    _idx=IDX[LABEL==prod_list[i]]
    np.random.seed(101)
    np.random.shuffle(_idx)
    test_idx.extend(_idx[:len(_idx)//10])
    val_idx.extend(_idx[len(_idx)//10 : len(_idx)//5])
    train_idx.extend(_idx[len(_idx)//5:])

100%|██████████| 8/8 [00:02<00:00,  2.71it/s]


In [52]:
all_idx=np.arange(g.number_of_nodes(ntype="usaanr"))

test_idx=np.array(test_idx)
val_idx=np.array(val_idx)
train_idx=np.array(train_idx)

train_mask=mask_func(train_idx,all_idx)
val_mask=mask_func(val_idx,all_idx)
test_mask=mask_func(test_idx,all_idx)

train_mask=torch.tensor(train_mask,dtype=bool)
val_mask=torch.tensor(val_mask,dtype=bool)
test_mask=torch.tensor(test_mask,dtype=bool)

print("{:<30}{:<10,}".format("dimension of training mask", torch.sum(train_mask).item()))
print("{:<30}{:<10,}".format("dimension of val mask", torch.sum(val_mask).item()))
print("{:<30}{:<10,}".format("dimension of test mask", torch.sum(test_mask).item()))

dimension of training mask    19,649,532
dimension of val mask         2,456,191 
dimension of test mask        2,456,187 


In [53]:
pd.DataFrame(vertices_v1.iloc[train_idx]["Prod_Trace"].value_counts(dropna=False)).reset_index().\
rename(columns={'index':'product','Prod_Trace':'count'}).style.format({'count':'{:,}'})

,product,count
0,0,"14,325,296"
1,6,"2,232,148"
2,4,"1,698,651"
3,5,"1,005,893"
4,1,"177,723"
5,2,"165,088"
6,7,"42,715"
7,3,"2,018"


In [54]:
pd.DataFrame(vertices_v1.iloc[val_idx]["Prod_Trace"].value_counts(dropna=False)).reset_index().\
rename(columns={'index':'product','Prod_Trace':'count'}).style.format({'count':'{:,}'})

,product,count
0,0,"1,790,662"
1,6,"279,019"
2,4,"212,331"
3,5,"125,737"
4,1,"22,215"
5,2,"20,636"
6,7,"5,339"
7,3,252


In [55]:
pd.DataFrame(vertices_v1.iloc[test_idx]["Prod_Trace"].value_counts(dropna=False)).reset_index().\
rename(columns={'index':'product','Prod_Trace':'count'}).style.format({'count':'{:,}'})

,product,count
0,0,"1,790,661"
1,6,"279,018"
2,4,"212,331"
3,5,"125,736"
4,1,"22,215"
5,2,"20,635"
6,7,"5,339"
7,3,252


In [56]:
g.nodes["usaanr"].data["train_mask"]=train_mask.unsqueeze(1)
g.nodes["usaanr"].data["val_mask"]=val_mask.unsqueeze(1)
g.nodes["usaanr"].data["test_mask"]=test_mask.unsqueeze(1)

#### Save Graph

In [57]:
graph_dir="/workspace/cjiang/eagle_project/CAP_graph/CAP_wihtout_zipcode/"
start=time.time()
with open(os.path.join(graph_dir,"CAP_Graph"),"wb") as f:
    pickle.dump((g,node_labels),f)
end=time.time()
print("It took {:0.4f} seconds to save graph database".format(end-start))

It took 21.1254 seconds to save graph database


In [2]:
graph_dir="/workspace/cjiang/eagle_project/CAP_graph/CAP_wihtout_zipcode/"
start=time.time()
with open(os.path.join(graph_dir,"CAP_Graph"),"rb") as f:
    G,node_labels=pickle.load(f)
end=time.time()
print("It took {:0.4f} seconds to load graph database".format(end-start))

It took 8.6530 seconds to load graph database


In [6]:
graph_show(G)

**************************************************
Node_types:  ['usaanr']
Edge_types:  ['AUTO_RELATED', 'Brother_Sister', 'Busi_rel_Other', 'Child', 'Ex-Spouse', 'Parent', 'Pers_rel_Other', 'SPONSEE', 'SPONSOR', 'Spouse', 'Step-Child', 'Step-Parent']
**************************************************
Canonical Etypes of Graph is:

usaanr              AUTO_RELATED        usaanr              
usaanr              Brother_Sister      usaanr              
usaanr              Busi_rel_Other      usaanr              
usaanr              Child               usaanr              
usaanr              Ex-Spouse           usaanr              
usaanr              Parent              usaanr              
usaanr              Pers_rel_Other      usaanr              
usaanr              SPONSEE             usaanr              
usaanr              SPONSOR             usaanr              
usaanr              Spouse              usaanr              
usaanr              Step-Child          usaanr          

In [67]:
torch.empty(3,2)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [71]:
C=torch.empty(G.num_nodes('usaanr'))
for etype in G.etypes:
    C+=G.in_degrees(etype=etype)
print("{:<35}{:<10,} ".format("The number of zero in-degree nodes is ",torch.sum(C==0)))

The number of zero in-degree nodes is 0          


#### Find metapaths in the generated heterogeneous graph

In [7]:
G.metagraph

<bound method DGLHeteroGraph.metagraph of Graph(num_nodes={'usaanr': 24561910},
      num_edges={('usaanr', 'AUTO_RELATED', 'usaanr'): 8056456, ('usaanr', 'Brother_Sister', 'usaanr'): 618666, ('usaanr', 'Busi_rel_Other', 'usaanr'): 1022064, ('usaanr', 'Child', 'usaanr'): 11067385, ('usaanr', 'Ex-Spouse', 'usaanr'): 3682614, ('usaanr', 'Parent', 'usaanr'): 11067385, ('usaanr', 'Pers_rel_Other', 'usaanr'): 1892856, ('usaanr', 'SPONSEE', 'usaanr'): 15958685, ('usaanr', 'SPONSOR', 'usaanr'): 15958685, ('usaanr', 'Spouse', 'usaanr'): 15485971, ('usaanr', 'Step-Child', 'usaanr'): 1165991, ('usaanr', 'Step-Parent', 'usaanr'): 1165991},
      metagraph=[('usaanr', 'usaanr', 'AUTO_RELATED'), ('usaanr', 'usaanr', 'Brother_Sister'), ('usaanr', 'usaanr', 'Busi_rel_Other'), ('usaanr', 'usaanr', 'Child'), ('usaanr', 'usaanr', 'Ex-Spouse'), ('usaanr', 'usaanr', 'Parent'), ('usaanr', 'usaanr', 'Pers_rel_Other'), ('usaanr', 'usaanr', 'SPONSEE'), ('usaanr', 'usaanr', 'SPONSOR'), ('usaanr', 'usaanr', 'Sp

In [8]:
import itertools

def get_all_possible_metapaths(g, K):
    possible_metapaths = []
    metagraph = g.metagraph
    # Iterate over all possible K-length sequences of all canonical edge types
    for metapath in itertools.product(g.canonical_etypes, repeat=K):
        # Check if the sequence indeed connects and forms a metapath.
        # If the destination node type of an edge type is not the same as the source
        # node type of the next edge type, then the sequence is not a valid metapath.
        is_valid = True
        previous_ntype = None
        for srctype, etype, dsttype in metapath:
            if previous_ntype is not None and srctype != previous_ntype:
                is_valid = False
                break
            previous_ntype = dsttype

        if is_valid:
            possible_metapaths.append(metapath)
    filtered_metapaths = []
    for metapath in possible_metapaths:
        result_g = dgl.metapath_reachable_graph(g, metapath)
        if result_g.number_of_edges() > 0:
            filtered_metapaths.append(metapath)
    return filtered_metapaths

def pretty_print_metapath(metapath):
    # This function just pretty-prints the metapath
    item_list = sum([['(' + etype[1] + ')', etype[2]] for etype in metapath], [])
    item_list.insert(0, metapath[0][0])
    return ' -- '.join(item_list)

In [9]:
for K in range(1, 3):
    print('### Length', K, 'metapaths ###')
    possible_metapaths = get_all_possible_metapaths(G, K)
    # print the metapaths
    for metapath in possible_metapaths:
        print(pretty_print_metapath(metapath))

### Length 1 metapaths ###
usaanr -- (AUTO_RELATED) -- usaanr
usaanr -- (Brother_Sister) -- usaanr
usaanr -- (Busi_rel_Other) -- usaanr
usaanr -- (Child) -- usaanr
usaanr -- (Ex-Spouse) -- usaanr
usaanr -- (Parent) -- usaanr
usaanr -- (Pers_rel_Other) -- usaanr
usaanr -- (SPONSEE) -- usaanr
usaanr -- (SPONSOR) -- usaanr
usaanr -- (Spouse) -- usaanr
usaanr -- (Step-Child) -- usaanr
usaanr -- (Step-Parent) -- usaanr
### Length 2 metapaths ###
usaanr -- (AUTO_RELATED) -- usaanr -- (AUTO_RELATED) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Brother_Sister) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Busi_rel_Other) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Child) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Ex-Spouse) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Parent) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (Pers_rel_Other) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (SPONSEE) -- usaanr
usaanr -- (AUTO_RELATED) -- usaanr -- (SPONSOR) -- usaanr
usaanr -

In [10]:
print("Total number of metapath(length 2) is {}".format(len(possible_metapaths)))

Total number of metapath(length 2) is 144
